In [ ]:
# import required libraries
from azure.ml import MLClient, dsl
from azure.ml.entities import CommandJob, Code, PipelineJob, ResourceConfiguration
from azure.ml.entities import JobInput, JobOutput, Dataset
from azure.identity import InteractiveBrowserCredential

In [ ]:
# enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
# get a handle to the workspace
ml_client = MLClient(credential=InteractiveBrowserCredential(), subscription_id=subscription_id, resource_group_name=resource_group, workspace_name=workspace)

## Create command jobs

In [ ]:
# define the job to get data
get_data_cmd = 'bash main.sh ${{outputs.cifar}}'
get_data_outputs = {"cifar": JobOutput()}
get_data_job = CommandJob(
    command = get_data_cmd,
    outputs = get_data_outputs,
    environment = 'AzureML-Minimal:18',
    compute = 'cpu-cluster',
    code = Code(local_path="./get-data")
)

In [ ]:
# define the job to train the model
train_model_cmd = 'python main.py --data-dir ${{inputs.cifar}} --epochs ${{inputs.epochs}} --model-dir ${{outputs.model_dir}}'
train_model_inputs = {'epochs': 1, 'cifar' : JobInput()}
train_model_outputs = {'model_dir': JobOutput()}
train_model_job = CommandJob(
    command = train_model_cmd,
    inputs = train_model_inputs,
    outputs=train_model_outputs,
    code = Code(local_path="./train-model"),
    distribution = {
        'distribution_type': 'PyTorch',
        'process_count_per_instance': 2},
    resources=ResourceConfiguration(instance_count=2),
    environment = 'AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu:6',
    compute = 'gpu-cluster'
)

In [ ]:
# define the job to evaluate the model
eval_model_cmd = 'python main.py --data-dir ${{inputs.cifar}} --model-dir ${{inputs.model_dir}}/model'
eval_model_inputs = {'cifar': JobInput(), 'model_dir': JobInput()}
eval_model_job = CommandJob(
    command = eval_model_cmd,
    inputs = eval_model_inputs,    
    code = Code(local_path="./eval-model"),
    distribution = {
        'distribution_type': 'PyTorch',
        'process_count_per_instance': 2},
     resources=ResourceConfiguration(instance_count=2),
    environment = 'AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu:6',
    compute = 'gpu-cluster'
)

## Build pipeline

In [ ]:
# convert jobs into components
get_data_func = dsl.load_component(component=get_data_job)
train_model_func = dsl.load_component(component=train_model_job)
eval_model_func = dsl.load_component(component=eval_model_job)

In [ ]:
@dsl.pipeline(
    name="hello_world_pipeline",
    compute="cpu-cluster",
    description="Basic Pipeline Job with 3 Hello World components",
)
def sample_pipeline():
    get_data = get_data_func()
    train_model = train_model_func(epochs=1, cifar=get_data.outputs.cifar)
    eval_model = eval_model_func(cifar=get_data.outputs.cifar, model_dir=train_model.outputs.model_dir)

pipeline = sample_pipeline()
    

In [ ]:
#submit the pipeline job
returned_job = ml_client.jobs.create_or_update(pipeline)
#get a URL for the status of the job
returned_job.services["Studio"].endpoint